In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import statsmodels.api as smi
import scipy.special as special
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px

from sklearn.linear_model import LinearRegression, LogisticRegression

def load_lottie(url):
    r = requests.get(url)
    if r.status_code == 200:
        return r.json()
    return None

@st.cache_resource
def load_data():
    df = pd.read_csv('Bozeman_V1.csv')
    numeric_df=df.select_dtypes(['float','int'])
    numeric_cols=numeric_df.columns

    text_df=df.select_dtypes(['object'])
    text_cols=text_df.columns
    return df, numeric_cols, text_cols

def load_data_logistic():
    # Cambia el nombre del archivo por el del dataset real para regresión logística
    df = pd.read_csv('Bozeman_V2.csv')
    numeric_df = df.select_dtypes(['float','int'])
    numeric_cols = numeric_df.columns.tolist()

    text_df = df.select_dtypes(['object'])
    text_cols = text_df.columns.tolist()
    return df, numeric_cols, text_cols
#en la view descripcion se muestra una descripcion general de la variable seleccionadad
#que significa, que tipos de valores tiene, etc 
#en la view regresion lineal simple se hace una regresion lineal simple entre dos variables, una dependiente y una independiente, y se muestra con un heatmap o una grafica de dispersión
#en la view regresion lineal multiple se hace una regresion lineal multiple entre varias variables, una dependiente y varias independientes y se muestra con un heatmap o una grafica de dispersión
#en la view regresion logistica se hace una regresion logistica entre dos variables, una dependiente y una independiente y se muestra con un heatmap o una grafica de dispersión
df, numeric_cols, text_cols = load_data()

st.sidebar.title("DASHBOARD")
st.sidebar.header("sidebar")
st.sidebar.subheader("Panel de selección")
view = st.sidebar.selectbox(
    label="Opciones", 
    options=['bienvenida', 'Descripcion', 'regresion lineal simple', 'regresion lineal multiple', 'regresion logistica'], 
    index=0
)

if view == 'bienvenida':
    # 1) Header con imagen de fondo y overlay de título
    st.markdown(
        """
        <style>
        .header {
            background-image: url('https://images.unsplash.com/photo-1500784215286-5b3bf986ae76');
            background-size: cover;
            background-position: center;
            height: 250px;
            position: relative;
            border-radius: 10px;
            margin-bottom: 1rem;
        }
        .header-title {
            position: absolute;
            bottom: 20px;
            left: 30px;
            color: white;
            font-size: 3rem;
            text-shadow: 2px 2px 8px rgba(0,0,0,0.7);
        }
        .metric-card {
            background: rgba(255,255,255,0.8);
            border-radius: 8px;
            padding: 1rem;
            text-align: center;
        }
        </style>
        <div class="header">
          <div class="header-title">¡Bienvenido a Bozeman! 🎉</div>
        </div>
        """,
        unsafe_allow_html=True
    )

    # 2) Métricas automáticas
    mcol1, mcol2, mcol3 = st.columns(3)
    with mcol1:
        st.markdown(f"<div class='metric-card'><h3>📊 Variables Num.</h3><h1>{len(numeric_cols)}</h1></div>", unsafe_allow_html=True)
    with mcol2:
        st.markdown(f"<div class='metric-card'><h3>🔤 Variables Cat.</h3><h1>{len(text_cols)}</h1></div>", unsafe_allow_html=True)
    with mcol3:
        st.markdown(f"<div class='metric-card'><h3>📑 Registros</h3><h1>{df.shape[0]}</h1></div>", unsafe_allow_html=True)

    st.markdown("---")

    # 3) Animación Lottie
    lottie_url = "https://assets9.lottiefiles.com/packages/lf20_jcikwtux.json"
    lottie_json = load_lottie(lottie_url)
    if lottie_json:
        st_lottie(lottie_json, height=200, key="welcome")

    st.markdown(
        """
        _Esta aplicación te guiará por las distintas técnicas de regresión utilizando  
        datos reales de Bozeman. En el menú lateral encontrarás:_

        - **🔍 Descripción de Variables**  
        - **📈 Regresión Lineal Simple**  
        - **📊 Regresión Lineal Múltiple**  
        - **🧮 Regresión Logística**  
        """
    )

    # 4) Botón interactivo para mostrar muestra de datos
    if st.button("👀 Ver muestra de datos"):
        st.dataframe(df.sample(5).reset_index(drop=True), use_container_width=True)

    st.markdown("---")

    # 5) Expander con curiosidades
    with st.expander("❓ Sabías que..."):
        st.write(
            "- Bozeman es la puerta de entrada al Parque Nacional Yellowstone.  \n"
            "- Tiene más de 200 días de sol al año.  \n"
            "- Su economía universitaria crece un 5% anual."
        )

if view == 'Descripcion':
    df, numeric_cols, text_cols = load_data()
    st.title('Descripcion de datos de Bozeman')
    # Se calcula la correlación entre las variables numéricas
    corr_matrix = df[numeric_cols].corr()
    fig_heatmap = px.imshow(
        corr_matrix,
        text_auto=True,
        aspect="auto",
        title="Matriz de Correlación",
        labels=dict(x="Variables", y="Variables")
    )
    #COLUMNAS que existen
    st.write('Columnas que existen en el dataset') 
    st.write(df.columns)   
    st.plotly_chart(fig_heatmap)
    st.title('Descripcion de la variable')
    selected_col = st.selectbox('selecciona una variable', numeric_cols)
    st.write('Descripcion de la variable', selected_col)
    st.write(df[selected_col].describe())
    st.write('Valores unicos de la variable', selected_col)
    st.write(df[selected_col].unique())
    #grafica de los datos, mostrarlo para que el usuario elija el tipo de grafico

    st.write('Grafica de los datos')
    fig = px.histogram(df, x=selected_col)
    st.plotly_chart(fig)

    st.write('Grafica de los datos con un grafico de pastel')
    fig = px.pie(df, values=selected_col, names=selected_col)
    st.plotly_chart(fig)

    #grafica de los datos con un heatmap
    #st.write('Grafica de los datos con un heatmap')
    #fig = px.imshow(df.corr(), text_auto=True)
    #st.plotly_chart(fig)
if view == 'regresion lineal simple':
    df, numeric_cols, text_cols = load_data()
    st.title('Regresion lineal simple')
    #st.subheader('Selecciona una variable dependiente')
    selected_col = st.selectbox('Selecciona una variable dependiente', numeric_cols)
    #st.subheader('Selecciona una variable independiente')
    selected_col2 = st.selectbox( 'Selecciona una variable independiente',numeric_cols)
    #regresion lineal simple
    st.write('Regresion lineal simple entre', selected_col, 'y', selected_col2)
    fig = px.scatter(df, x=selected_col, y=selected_col2)
    st.plotly_chart(fig)
if view == 'regresion lineal multiple':
    df, numeric_cols, text_cols = load_data()
    st.title('Regresion lineal multiple')
    #st.subheader('Selecciona una variable dependiente')
    selected_col = st.selectbox('Selecciona una variable dependiente', numeric_cols)
    #st.subheader('Selecciona una variable independiente')
    selected_col2 = st.multiselect( 'Selecciona variables independientes',numeric_cols)
    #regresion lineal simple
    st.write('Regresion lineal multiple entre', selected_col, 'y', selected_col2)
    fig = px.scatter_matrix(df, dimensions=[selected_col]+selected_col2)
    st.plotly_chart(fig)
if view == 'regresion logistica':
    df_log, numeric_cols_log, text_cols_log = load_data_logistic()
    st.title('Regresión Logística')
    # Permite que el usuario elija de entre todas las columnas para la dependiente
    dependent = st.selectbox('Selecciona la variable dependiente (binaria)', df_log.columns.tolist())
    independent = st.selectbox('Selecciona la variable independiente (numérica)', numeric_cols_log)
    st.write('Regresión logística entre', dependent, 'y', independent)
    
    # Validar que la variable dependiente sea binaria
    if df_log[dependent].nunique() != 2:
        st.error("La variable dependiente seleccionada no es binaria. Por favor, selecciona otra variable.")
    else:
        X = df_log[[independent]]
        y = df_log[dependent]
        # Convertir a valores numéricos si es de tipo objeto
        model_log = LogisticRegression()
        model_log.fit(X, y)
        st.write("Modelo entrenado correctamente.")
        st.write("Coeficiente:", round(model_log.coef_[0][0], 2))
        st.write("Intersección:", round(model_log.intercept_[0], 2))
        
        # Visualización: aunque en regresión logística se suele usar otras métricas, mostramos un scatter
        fig_log = px.scatter(df_log, x=independent, y=dependent, title="Scatter de Regresión Logística")
        st.plotly_chart(fig_log)









ModuleNotFoundError: No module named 'streamlit_lottie'

: 